# First review the base knowage of linear Algebra

<a href="https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.linalg.eig.html">eig</a> 

In [1]:
import numpy as np

In [2]:
np.random.seed(0)
A=np.random.rand(5,5)*5;
print(A)
print("------------------")
W,S=np.linalg.eig(A)
S_inv=np.linalg.inv(S)
B=S.dot(np.diag(W)).dot(S_inv)
print(B.real)

[[ 2.74406752  3.57594683  3.01381688  2.72441591  2.118274  ]
 [ 3.22947057  2.18793606  4.458865    4.8183138   1.91720759]
 [ 3.95862519  2.6444746   2.84022281  4.62798319  0.35518029]
 [ 0.4356465   0.10109199  4.16309923  3.89078375  4.35006074]
 [ 4.89309171  3.99579282  2.30739681  3.90264588  0.59137213]]
------------------
[[ 2.74406752  3.57594683  3.01381688  2.72441591  2.118274  ]
 [ 3.22947057  2.18793606  4.458865    4.8183138   1.91720759]
 [ 3.95862519  2.6444746   2.84022281  4.62798319  0.35518029]
 [ 0.4356465   0.10109199  4.16309923  3.89078375  4.35006074]
 [ 4.89309171  3.99579282  2.30739681  3.90264588  0.59137213]]


# next we implement PR algrithm
formula: pr_new=(1-d)/n+d* (W*pr_old) <br />
d is prob that a user keep stay at remain sites(prob a given vertex jump to another vertex)<br />
W[i][j]=I{j-->i}/degree(j)

In [3]:
def PR(W,prob=0.85,t=10):
    '''
        W:relation matrix,wij=I[j->i] / degree(j)
        prob:prob jump from one to another vertex
        t:number of iteration
        return:rank,rank score,decent order
    '''
    
    n=W.shape[0]
    #initial uniformly
    init_score=np.ones(n)/n
    
    E,S=np.linalg.eig(W)
    S_inv=np.linalg.inv(S)
    
    #resolve or project initial score to eig space S
    resolve=S_inv.dot(init_score)
    
    score=np.real(S.dot(np.diag(E**t)).dot(resolve))
    score=prob*score+(1-prob)/n
    
    
    rank=np.argsort(-score)
    rankScore=score[rank]
    return rank,rankScore

### example from this video <a href='https://www.youtube.com/watch?v=P8Kt6Abq_rM'>video</a> 

In [ ]:
W=np.array([
        [0,0,1/3,0],
        [1/2,0,1/3,0],
        [1/2,0,0,1],
        [0,1,1/3,0]
    ])
PR(W,prob=1,t=99)

## Now review some NLTK 
## TOKENIZE
## Corpara: body of text
## Lexicon:the mean of word

In [ ]:
from nltk import sent_tokenize,word_tokenize
#unfortunally this fail to tokenize chinere,using jieba instead
import jieba

In [ ]:
msg="黄晓明工作室对此发声明否认，称黄晓明不认识高某，未参与过任何操纵股票的行为；黄晓明未曾受过任何与股票有关的处罚，亦未介入过任何与股票有关的调查。黄晓明工作室还敦促相关方即刻删撤一切虚假信息，以免对公众造成误导并导致己方法律责任的承担。返回搜狐"

In [ ]:
list(jieba.cut(msg,cut_all=False))

In [30]:
import jieba.posseg as pseg

In [31]:
words=[]
for char,pos in pseg.cut(msg):
#     if pos.startswith('n') or pos.startswith('a'):
    print('word %s,pos:%s'%(char,pos))
#         words.append(char)
words=set(words)

NameError: name 'msg' is not defined

# some document about jieba <a href='https://github.com/fxsjy/jieba'>doc</a>

In [32]:
def corpora2Sentence(corpos):
    '''
        cut corpos into sentences
        return:
            sentences:list,one element represent a none adj,select[i]=(word,pos) mean word as pos
            vocabs:a set of all words occur in corpos
            word_idx and idx_word:the map between idx and vocab,I using a matrix representation
                for the graph
            vocabs_position:a dict,the element look like: 
                V(i):[p1,p2,..pn],mean word V occur at positon p1,p2,..pn
            position_vocabs:reverse of vocabs_position:
                pi:V(j),mean at positon pi is word V(j)
    '''
    sentences=[]
    vocabs=[]  #store all vocabulary
    vocabs_position={}
    word_idx=0 #the word position at corpos
    for char,pos in pseg.cut(corpos):
        word_idx+=1
        if pos=='x':continue
        elif (pos.startswith('n') or pos=='a' or pos.startswith('v'))and len(char)>1:
            sentences.append((char,word_idx))
            vocabs.append(char)
            if char in vocabs_position:vlist=vocabs_position[char]
            else:
                vlist=[]
                vocabs_position[char]=vlist
            vlist.append(word_idx)
        
        
    vocabs=set(vocabs)
    word_idx={c:i for i,c in enumerate(vocabs)}
    idx_word={i:c for i,c in enumerate(vocabs)}
    
    position_vocabs={}
    for vocb,poslist in vocabs_position.items():
        for p in poslist:position_vocabs[p]=vocb

    return sentences,vocabs,word_idx,idx_word,vocabs_position,position_vocabs

## Now I need a little Graph Theory to handle the output

In [24]:
#[产品]-[]
#[信息化]-[]
#[商用车]-[]
#[工业]-[]
#[新能源]-[汽车,船舶]
#[汽车]-[产品,标准]
#[船舶]-[标准]
#[标准]-[]
#[目录]-[]
#[车船税]-[]
#output should be  信息化 商用车 工业 新能源-汽车-产品 新能源-汽车-标准 新能源-船舶-标准 目录 车船税

In [25]:
class Vertex():
    '''
        I using adjacency list representations,a convinient way to do DFS
    
    '''
    def __init__(self,w):
        self.content=w
        self.inDegree=0
        self.adj=set() #adjacentcy vertexs
    def __eq__(self,other):
        return self.content==other.content
    def __repr__(self):
        return self.content
    def __str__(self):
        return self.__repr__()
    def __hash__(self):
        return hash(self.content)
class Vsets():
    def __init__(self,words):
        '''
            maintain a set contain all the vertex
        '''
        self.vertexes={w:Vertex(w) for w in words}
    def connect(self,v1,v2):
        '''
            add a edge from v1->v2
            v1,v2 are string(key of self.vertexes)
        '''
        v1=self.vertexes[v1]
        v2=self.vertexes[v2]
        v1.adj.add(v2)
        v2.inDegree+=1
    def DFS(self):
        '''
            Do Depth First Serch on vertexes,start point is all vertex that
            have inDegree of 0,in a other word,it's the root of tree
            
            return: a list of all the possible combination of content from root to leaf
        '''
        
        #a helper dict member all visited vertex,initial state:no one be visited
        self.visited={}
        #record all vertex have in degress 0
        root=set()
        for w,v in self.vertexes.items():
            self.visited[v]=False
            if v.inDegree==0:
                root.add(v)
                
                
        
        ret=[]
        #do DFS_visit from root
        for v in root:
            v_list=self.DFS_Visit(v)
            ret.extend(v_list)
        del self.visited
        return ret
    def DFS_Visit(self,s):
        '''
            start from s,DFS to leaf
            return: a list of all the possible combination of content from s to it's leaf
        '''
        self.visited[s]=True #first label this a visited vertex
        ret=[]
        
        
        for u in s.adj:
            if self.visited[u]==False:
                u_list=self.DFS_Visit(u)  #a list of all possible from u,do DFS_VISIT
            else:
                u_list=self.visited[u] #no need search 
            #append prefix s.content to subTree
            ret.extend([s.content+c for c in u_list])
        
        #if s have no subTree to visited
        if len(ret)==0:ret.append(s.content)
        self.visited[s]=ret
        return sorted(ret)
#[产品]-[]
#[信息化]-[]
#[商用车]-[]
#[工业]-[]
#[新能源]-[汽车,船舶]
#[汽车]-[产品,标准]
#[船舶]-[标准]
#[标准]-[]
#[目录]-[]
#[车船税]-[]
#output should be  信息化 商用车 工业 目录 车船税 新能源-汽车-产品 新能源-汽车-标准 新能源-船舶-标准 

vset=Vsets(['产品','信息化','商用车','工业','新能源','汽车','船舶','标准','目录','车船税'])
vset.connect('新能源','汽车')
vset.connect('新能源','船舶')
vset.connect('汽车','产品')
vset.connect('汽车','标准')
vset.connect('船舶','标准')
# vset.connect('新能源','汽车')
print(vset.DFS())

['工业', '新能源汽车产品', '新能源汽车标准', '新能源船舶标准', '商用车', '目录', '信息化', '车船税']


# Construct Sequence Graph Model

In [26]:
class Graph():
    def __init__(self,filename):
        '''
        
        '''
        #cut corpus to words,remain only none and adjective
        with open(filename) as f:
            corpora=f.read()
            self.sentence,self.vocab,self.word_idx,self.idx_word,self.vocabs_position,self.position_vocabs=corpora2Sentence(corpora)
        
        self.words=len(self.vocab)
        self.H=np.zeros((self.words,self.words)) #matrix representation
        
    def _connect(self,char1,char2):
        '''
            char1,char2 is select 2 words by jieba,
            char2 is after char2,
            if the distance between char1,char2 is less than window_size,
            I treat this is a valid edge,then update graph matrix
            
            char1:tuple(word1,position1)
            
            return True/False
        '''
        c1,p1,c2,p2=*char1,*char2
        
        if(abs(p2-p1)>self.windowSize):return False
        idx1,idx2=self.word_idx[c1],self.word_idx[c2]
        if idx1!=idx2:self.H[idx1,idx2]=self.H[idx2,idx1]=1
        return True
    def getMatrix(self,windowSize=5,eps=1e-6):
        '''
            compute graph matrix,using by Page Rank 
            windowsSize: within windowsSize ragime of word s view as neibourhood of s
        '''
        self.windowSize=windowSize
        
        sentencelen=len(self.sentence)
        for i in range(sentencelen):
            char1=self.sentence[i]
            for j in range(i+1,sentencelen):
                char2=self.sentence[j]
                if self._connect(char1,char2)==False:break
        
        degree=np.sum(self.H,axis=0)
        degree[degree==0]=eps
        H=self.H/degree
        return H
    def rank(self,indices,top=1000):
        '''
            indices:the word's index according to Page Rank Algrithm,in decent order
            top:select first top word of indices
        '''

        
        selectedWords=[]
        for i in range(min(len(indices),top)):
            selectedWords.append(self.idx_word[indices[i]])
        #using graph tool to handle the output
        vstool=Vsets(selectedWords)
        
        #sort all positon of selected word occur in corpos
        sel_pos_list=[]
        for w in selectedWords:
            sel_pos_list.extend(self.vocabs_position[w])
        #sel_pos_list is a list of selectWords's positon ,for example,个税 is a select word,
        #occur at position 10,30(reference to corpos),sel_pos_list look like [....10...30.....]
        sel_pos_list=sorted(sel_pos_list) 
        
        
        '''
            no go through sel_pos_list, if sel_pos_list[i] sel_pos_list[i+1] are continue posion ,
            that mean W[sel_pos_list[i]] and W[sel_pos_list[i+1]] should merged as one word,
            with vstool to add a edge from  W[sel_pos_list[i]] to W[sel_pos_list[i+1]],
            vstool can handle consitive word properly!
        
        '''
        for i in range(len(sel_pos_list)-1):
            pi=sel_pos_list[i]
            Wi=self.position_vocabs[pi]
            pj=sel_pos_list[i+1]
            Wj=self.position_vocabs[pj]
            if(pj-pi==1 and Wi !=Wj):
                vstool.connect(Wi,Wj)
        ret=vstool.DFS()
        return ret

In [47]:
graph=Graph('tx4.txt')

In [48]:
H=graph.getMatrix(10)
print(graph.vocab)

{'大众', '退税', '新华社', '公众', '对象', '退库', '教学大纲', '具有', '缴纳', '倡导', '广西', '财政部', '小学', '传媒', '进行', '包括', '机关', '有关', '财税', '图书', '通知', '零售', '简称', '后退', '军事', '教育', '自治区', '免税', '出现', '职业高中', '国家税务总局', '软盘', '少数民族', '注释', '具体', '掌握', '专业', '出版发行', '详见', '任务', '基地', '简并', '取得', '报纸', '含以', '财政', '行政', '政策', '指以', '辅导', '科学', '主管', '许可证', '宁夏', '传播', '期刊', '审定', '印刷', '办事处', '列入', '相应', '核算', '光盘', '制作', '部门', '开放', '参考书', '共青团', '相关', '申请', '环节', '印发', '目录', '民主党派', '利用', '延续', '政府', '练习册', '参与', '新疆维吾尔自治区', '载体', '课外读物', '营业税', '组织', '党政部门', '信息', '出版单位', '增值税', '主管部门', '名单', '分离', '优惠政策', '批准', '预字', '地震台', '批发', '追回', '方式', '理解', '规定', '变更', '办理', '内蒙古', '新疆', '注册', '普通', '残联', '知识', '行政部门', '出版物', '租型', '纳税人', '附件', '门票', '改革', '征税', '教材', '弘扬', '中等职业', '业务', '活动', '形式', '照章', '监察', '原因', '税收', '财务', '实行', '问题', '自读', '税务机关', '货物', '重组', '废止', '无法', '定义', '单位', '核准', '图册', '出版', '专员', '下列', '科学技术', '免征', '成人', '部分', '学生上课', '中国共产党', '工会', '享受', '博物馆', '征收', '起至', '管理', '入库', '开具', '电子',

In [49]:
rank,rankScore=PR(H)
print(graph.rank(rank,10))
# print(rankScore)
#['产品', '信息化', '商用车', '工业', '新能源', '标准', '汽车', '目录', '船舶', '车船税']

['课本', '单位', '报纸', '出版物出版', '通知', '期刊增值税先征后退']
